In [1]:
import urllib2
from bs4 import BeautifulSoup as bs
import re
import yelp
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator
import json
import re
import pandas as pd

# Scrape Restaurant data from the Washingtonian website

In [2]:
main = "http://www.washingtonian.com/projects/dining/100-very-best-restaurants/2015/"
soup = bs(urllib2.urlopen(main),'lxml')
all_n = soup.findAll("span", {"class":"name"})
all_names = [n.get_text() for n in all_n]

# clean with re
clean_names = [re.sub(r"^#\d{1,3}\s","",item)for item in all_names]
clean_names = [re.sub(r"\xe9","e",item) for item in clean_names]
clean_names = [re.sub(r"\xf1","n",item) for item in clean_names]

In [3]:
rows = soup.findAll("tr")

prices = [i.findAll("td")[2].get_text() for i in rows]

In [4]:
prices = [re.sub(r"\xa2","$",item)for item in prices]

# Sign on to Yelp API

In [5]:
    
auth = Oauth1Authenticator(
    consumer_key = "eowiz8Zwiat1Xk7xTdJ_8w",
    consumer_secret = "MQSNeB1_jBUqB-vZlttsDHoI0Ys",
    token = "qqSlJgWqLVyEB06bWQOYgUpJLdvCVXJy",
    token_secret = "hvdR-gd7QvR3sojquwIVzroNC4A"
)

client = Client(auth)

client

# Get YELP data of top 100 restuarants in Washington.
From the Washingtonian Top 100 


In [6]:
params = {
    'term': 'food',
    'lang': 'fr'
}

client.search('San Francisco', **params)

### Fix for restaurants that are wrong in search

In [7]:
bus_info1 = []

for item in clean_names:
    if item == "PassionFish":
        params = {'lang': 'eng', 'term':item, 'limit':1}
        response = client.search('Reston', **params)
        bus_info1.append(response)
    elif item == "Vin 909 Winecafe":
        params = {'lang': 'eng', 'term': 'Vin 909', 'limit':1}
        response = client.search('Washington DC', **params)
        bus_info1.append(response)
    elif item == "The Inn at Little Washington":
        params = {'lang': "eng", 'term': 'Inn Little Washington', 'limit':1}
        response = client.search('Washington VA', **params)
        bus_info1.append(response)
    else:
        params = {'lang': "eng", "term":item, "limit":1}
        response = client.search("Washington DC", **params)
        bus_info1.append(response)

In [8]:
bus_data = []
for num in range(len(bus_info1)):
    x = bus_info1[num].businesses[0]
    b_name = x.name
    b_rating = x.rating.rating
    b_loc_long = x.location.coordinate.longitude
    b_loc_lat = x.location.coordinate.latitude
    b_street = ' '.join(x.location.address)
    b_city = x.location.city
    b_state = x.location.state_code
    b_postal = x.location.postal_code
    b_neighborhood = "NA" if x.location.neighborhoods is None else x.location.neighborhoods[0]
    b_review_count = x.review_count
    b_categories = x.categories[0][0]
    b_address = b_street +", "+b_city + ", " + b_state + " " + b_postal
    
    bus_data.append({"name":b_name,"rating":b_rating, "long":b_loc_long,"lat":b_loc_lat,
    "address":b_address,
    "street":b_street,
    "city":b_city,
    "state":b_state,
    "postal":b_postal,
    "neighborhood":b_neighborhood,
    "review_count":b_review_count,
    "categories":b_categories,
    "price":prices[num]})

# Save as DataFrame and replace errors in unicode

In [9]:
restaurants = pd.DataFrame(bus_data)

In [10]:
restaurants

,address,categories,city,lat,long,name,neighborhood,postal,price,rating,review_count,state,street
0,"1509 17th St NW, Washington, DC 20036",Greek,Washington,38.910122,-77.038132,Komi,Dupont Circle,20036,$$$,4.5,568,DC,1509 17th St NW
1,"855 E St NW, Washington, DC 20004",American (New),Washington,38.896263,-77.023499,minibar by José Andrés,Penn Quarter,20004,$$$,4.5,210,DC,855 E St NW
2,"717 8th St SE, Washington, DC 20003",American (New),Washington,38.880638,-76.995338,Rose's Luxury,Capitol Hill,20003,$$,4.5,868,DC,717 8th St SE
3,"3050 K St NW Ste 101, Washington, DC 20007",Seafood,Washington,38.902230,-77.060564,Fiola Mare,NA,20007,$$$,4.0,355,DC,3050 K St NW Ste 101
4,"633 D St NW, Washington, DC 20004",Indian,Washington,38.895008,-77.021286,Rasika,Penn Quarter,20004,$$,4.5,1892,DC,633 D St NW
5,"1511 17th St NW, Washington, DC 20036",Thai,Washington,38.910118,-77.038132,Little Serow,Dupont Circle,20036,$$,4.0,412,DC,1511 17th St NW
6,"1822 1st St NW, Washington, DC 20002",Italian,Washington,38.914837,-77.012172,The Red Hen,Bloomingdale,20002,$,4.0,336,DC,1822 1st St NW
7,"601 Pennsylvania Ave NW, Washington, DC 20004",Italian,Washington,38.894039,-77.020794,Fiola,Penn Quarter,20004,$$$,4.0,389,DC,601 Pennsylvania Ave NW
8,"42461 Lovettsville Rd, Lovettsville, VA 20180",American (Traditional),Lovettsville,39.280330,-77.556500,The Restaurant at Patowmack Farm,NA,20180,$$$,4.0,95,VA,42461 Lovettsville Rd
9,"909 Bay Ridge Ave, Annapolis, MD 21403",American (New),Annapolis,38.966750,-76.488460,VIN 909,NA,21403,$,4.5,299,MD,909 Bay Ridge Ave


In [11]:
restaurants.name = restaurants.name.str.replace(r"\xe9","e")
restaurants.name = restaurants.name.str.replace(r"'","")

In [12]:
restaurants.set_index("name")

,address,categories,city,lat,long,neighborhood,postal,price,rating,review_count,state,street
name,,,,,,,,,,,,
Komi,"1509 17th St NW, Washington, DC 20036",Greek,Washington,38.910122,-77.038132,Dupont Circle,20036,$$$,4.5,568,DC,1509 17th St NW
minibar by Jose Andres,"855 E St NW, Washington, DC 20004",American (New),Washington,38.896263,-77.023499,Penn Quarter,20004,$$$,4.5,210,DC,855 E St NW
Roses Luxury,"717 8th St SE, Washington, DC 20003",American (New),Washington,38.880638,-76.995338,Capitol Hill,20003,$$,4.5,868,DC,717 8th St SE
Fiola Mare,"3050 K St NW Ste 101, Washington, DC 20007",Seafood,Washington,38.902230,-77.060564,NA,20007,$$$,4.0,355,DC,3050 K St NW Ste 101
Rasika,"633 D St NW, Washington, DC 20004",Indian,Washington,38.895008,-77.021286,Penn Quarter,20004,$$,4.5,1892,DC,633 D St NW
Little Serow,"1511 17th St NW, Washington, DC 20036",Thai,Washington,38.910118,-77.038132,Dupont Circle,20036,$$,4.0,412,DC,1511 17th St NW
The Red Hen,"1822 1st St NW, Washington, DC 20002",Italian,Washington,38.914837,-77.012172,Bloomingdale,20002,$,4.0,336,DC,1822 1st St NW
Fiola,"601 Pennsylvania Ave NW, Washington, DC 20004",Italian,Washington,38.894039,-77.020794,Penn Quarter,20004,$$$,4.0,389,DC,601 Pennsylvania Ave NW
The Restaurant at Patowmack Farm,"42461 Lovettsville Rd, Lovettsville, VA 20180",American (Traditional),Lovettsville,39.280330,-77.556500,NA,20180,$$$,4.0,95,VA,42461 Lovettsville Rd


# Load table into MYSQL

In [13]:
import datetime
import MySQLdb as myDB
import warnings
import numpy as np

#def write2Mysql(Final_Table):

#Inserts dataframe into MySQL
con = myDB.connect(host='localhost', user='root', passwd='root')
cursor = con.cursor()
warnings.simplefilter("ignore")
cursor.execute("Create Database if not exists GroupProj;")
con.select_db("GroupProj")


In [14]:
restaurants.to_sql(con = con, flavor = "mysql", name = "restaurants", if_exists="replace")

# Check Data

In [15]:
cursor.execute("SHOW TABLES")

cursor.fetchall()

(('restaurants',),)

In [33]:
cursor.execute("SELECT * FROM restaurants LIMIT 1")
cursor.fetchall()

((0L,
  '1509 17th St NW, Washington, DC 20036',
  'Greek',
  'Washington',
  38.9101219177246,
  -77.0381317138672,
  'Komi',
  'Dupont Circle',
  '20036',
  '$$$',
  4.5,
  567L,
  'DC',
  '1509 17th St NW'),)